## Creating a `Dataset` object from text files

In [1]:
import pandas as pd
import cnmfsns as cn

data = pd.read_table("diamandis_data/diamandis_nobulk.txt", index_col=0)  # normalized expression data
metadata = pd.read_table("diamandis_data/diamandis_nobulk.metadata.txt", index_col=0)  # sample metadata

# create dataset from DataFrames
dataset = cn.Dataset.from_df(data=data, obs=metadata, is_normalized=True)


/home/tbverhey/miniconda3/lib/python3.9/site-packages/anndata/utils.py:143: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")
/home/tbverhey/github/cNMF-SNS/src/cnmfsns/core.py:109: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=data, obs=obs, var=var, uns=uns)


## Reading an writing `Dataset` objects

In [2]:
dataset.write_h5ad("diamandis.h5ad")  # write to .h5ad file

dataset = cn.Dataset.from_h5ad("diamandis.h5ad")  # read from .h5ad file 

cNMF-SNS can recognize AnnData .h5ad files whether they are created in [Seurat](https://satijalab.org/seurat/archive/v2.4/conversion_vignette.html), [scanpy](https://scanpy.readthedocs.io/en/stable/usage-principles.html#anndata), or other single-cell software tools.

In [3]:
import scanpy

pbmc = scanpy.datasets.pbmc3k()
pbmc_dataset = cn.Dataset.from_anndata(pbmc)

## Working with `Dataset` objects

For those interfacing with Scanpy or other tools, you can directly access the AnnData object like this:

In [4]:
dataset.compute_gene_stats()

AttributeError: module 'cnmfsns' has no attribute 'cnmf'

In [ ]:
cn.odg.__dir__()

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__file__',
 '__cached__',
 '__builtins__',
 'np',
 'logging',
 'pd',
 'sns',
 'plt',
 'ECDF',
 'Line2D',
 'Patch',
 'cn',
 'odg_plots',
 'fetch_hgnc_protein_coding_genes']

In [6]:
import cnmfsns as cn

<module 'cnmfsns.odg' from '/home/tbverhey/github/cNMF-SNS/src/cnmfsns/odg.py'>